# Implementasi Konvolusi 1D dari Nol dengan JAX

Notebook ini menjelaskan cara mengimplementasikan fungsi `jnp.convolve` dari tingkat dasar (scratch) menggunakan `jax.numpy`. Kita akan fokus pada logika matematika konvolusi dan bagaimana memanfaatkan vektorisasi JAX (`vmap`) untuk membuatnya efisien.

## 1. Definisi Matematika

Konvolusi satu dimensi antara dua urutan $a$ dan $v$ didefinisikan sebagai:

$$(a * v)[n] = \sum_{m=-\infty}^{\infty} a[m] v[n - m]$$

Dalam praktiknya (pemrosesan sinyal digital):
1. Membalik (reverse) sinyal kernel $v$.
2. Menggeser kernel yang dibalik tersebut di atas sinyal input $a$.
3. Pada setiap posisi, hitung *dot product* antara kernel dan jendela (window) input yang bersesuaian.

In [1]:
import jax
import jax.numpy as jnp
import time

## 2. Implementasi dari Nol

Kita akan membuat fungsi `convolve_scratch` yang mendukung mode standard: `full`, `same`, dan `valid`.

In [3]:
# Mari perbaiki mode 'same' agar lebih akurat mengikuti numpy
def convolve_scratch(a, v, mode='full'):
    """
    1D Convolution from scratch using JAX

    Args:
        a (array): Input array
        v (array): Kernel array
        mode (str): Mode of convolution ('full', 'same', 'valid')

    Returns:
        array: Convolved array
    """
    v_flipped = v[::-1]
    N, M = len(a), len(v)
    
    # Selalu mulai dari 'full' padding untuk kemudahan
    a_padded = jnp.pad(a, (M-1, M-1))
    
    if mode == 'full':
        out_len = N + M - 1
        start_idx = 0
    elif mode == 'same':
        out_len = N
        start_idx = (M - 1) // 2
    elif mode == 'valid':
        out_len = N - M + 1
        start_idx = M - 1
        
    indices = jnp.arange(out_len)
    def get_dot(i):
        window = jax.lax.dynamic_slice_in_dim(a_padded, i + start_idx, M)
        return jnp.dot(window, v_flipped)
    
    return jax.vmap(get_dot)(indices)

## 2.1 Memahami `jax.vmap` (Vectorizing Map)

Dalam implementasi `convolve_scratch` di atas, kita menggunakan `jax.vmap`. Ini adalah salah satu fitur paling kuat di JAX. Mari kita bahas lebih dalam:

### Apa itu `vmap`?
`vmap` singkatan dari **Vectorizing Map**. Secara sederhana, `vmap` mengambil sebuah fungsi yang dirancang untuk bekerja pada satu data (skalar atau array tunggal) dan mengubahnya secara otomatis menjadi fungsi yang bekerja pada seluruh kumpulan (batch) data secara paralel.

### Mengapa kita menggunakannya di sini?
Dalam konvolusi tradisional, kita biasanya menggunakan loop `for` untuk menggeser jendela kernel di atas input:
```python
for i in range(out_len):
    result[i] = jnp.dot(input[i:i+M], filter)
```
Loop `for` di Python sangat lambat untuk operasi numerik besar. `jax.vmap` menghilangkan kebutuhan akan loop ini dengan **vectorization**:
1. **Efektivitas XLA**: `vmap` mendelegasikan iterasi ke kompiler XLA (Accelerated Linear Algebra) yang dapat mengoptimalkan operasi tersebut untuk dijalankan di CPU, GPU, atau TPU.
2. **Simplisitas**: Kita hanya perlu menulis logika untuk satu elemen (`get_dot(i)`), dan JAX menangani sisanya.

### Bagaimana cara kerjanya di fungsi `convolve_scratch`?
1. Mendefinisikan `get_dot(i)` yang menghitung hasil untuk satu indeks `i` tertentu.
2. Mendefinisikan `indices = jnp.arange(out_len)` yang berisi daftar semua indeks yang ingin kita hitung.
3. Dengan `jax.vmap(get_dot)(indices)`, JAX akan memetakan `get_dot` ke setiap nilai dalam `indices` secara efisien dan mengembalikan hasilnya sebagai satu array utuh.

## 3. Verifikasi

Mari kita bandingkan hasilnya dengan `jnp.convolve` bawaan.

In [6]:
a = jnp.array([1, 2, 3, 4, 5])
v = jnp.array([1, 0, -1])

for mode in ['full', 'same', 'valid']:
    start_t = time.time()
    res_jax = jnp.convolve(a, v, mode=mode)
    elapsed_t_jax = time.time() - start_t

    start_t = time.time()
    res_scratch = convolve_scratch(a, v, mode=mode)
    elapsed_t_scratch = time.time() - start_t

    print(f"res_jax ({elapsed_t_jax:.4f}s): {res_jax}")
    print(f"res_scratch ({elapsed_t_scratch:.4f}s): {res_scratch}")

    match = jnp.allclose(res_jax, res_scratch)
    print(f"Mode: {mode:5} | Match: {match} | Result: {res_scratch}")

res_jax (0.0001s): [ 1.  2.  2.  2.  2. -4. -5.]
res_scratch (0.0024s): [ 1  2  2  2  2 -4 -5]
Mode: full  | Match: True | Result: [ 1  2  2  2  2 -4 -5]
res_jax (0.0000s): [ 2.  2.  2.  2. -4.]
res_scratch (0.0023s): [ 2  2  2  2 -4]
Mode: same  | Match: True | Result: [ 2  2  2  2 -4]
res_jax (0.0000s): [2. 2. 2.]
res_scratch (0.0017s): [2 2 2]
Mode: valid | Match: True | Result: [2 2 2]
